In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from openai import OpenAI

In [ ]:
client = OpenAI(api_key="your_key")


In [3]:
def clean_html(raw_html):
    if pd.isna(raw_html):
        return ""
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text(separator=" ", strip=True)

In [4]:
train_df = pd.read_csv(r"C:\\Users\\lu17w\\Downloads\\job_data_files\\salary_labelled_development_set.csv")  

In [5]:
def row_to_text(row, include_label=True):
    job_title = row["job_title"]
    # job_details = row["job_ad_details"]
    job_details = clean_html(row["job_ad_details"])

    location = row["nation_short_desc"]

    base_text = f"Job Title: {job_title}\nJob Description: {job_details}\nLocation: {location}"
    if include_label:
        base_text += f"\ny_true: {row['y_true']}"
    return base_text

In [6]:
shots = "\n\n".join([f"example{i+1}：\n{row_to_text(train_df.iloc[i])}" for i in range(10)])

In [7]:
test_df = pd.read_csv("C:\\Users\\lu17w\\Downloads\\job_data_files\\salary_labelled_test_set.csv")

test_df["predicted"] = ""
test_df["true_label"] = test_df["y_true"].astype(str)
results = []
correct = 0

for i, row in test_df.iterrows():
    query = f"query：\n{row_to_text(row, include_label=False)}\nyour_predict：y_true ="

    user_prompt = (
        "You are a classification assistant. Below are some job descriptions along with their corresponding salaries (y_true)：\n\n"
        "Please only give me answer, no more other things. The structure should be like 100-200-AUS-MONTHLY. (No commas are required between numbers)"
        "If you think there is no mention of salary, please give me 0-0-None-None."
        "Do not give me things like y_true = 0-0-None-None, only give me answer, like 0-0-None-None"
        # f"{shots}\n\n{query}"
        f"{query}"
    )

    messages = [
        {"role": "user", "content": user_prompt}
    ]

    response = client.chat.completions.create(
        model="ft:gpt-4.1-mini-2025-04-14:personal:6713:BQuQ64ON",
        messages=messages,
        max_tokens=30,
        temperature=0.7
    )

    prediction = response.choices[0].message.content.strip().split("\n")[0]

    results.append({
        "index": i,
        "predicted": prediction,
        "true_label": str(row["y_true"])
    })
    if prediction == str(row["y_true"]):
        correct += 1
    if (i + 1) % 100 == 0:
        checkpoint_df = pd.DataFrame(results)
        filename = f"gpt4mini/gpt-4.1-mini-salary_predictions_nohtmlcustom.csv"
        checkpoint_df.to_csv(filename, index=False)
        print(f"{i+1}, save result to：{filename}")
    number = i+1
    # if i>1:
    #     break

filename = f"gpt4mini/gpt-4.1-mini-salary_predictions_nohtmlcustom.csv"
final_df = pd.DataFrame(results)
final_df.to_csv(filename, index=False)

100, save result to：gpt4mini/gpt-4.1-mini-salary_predictions_nohtmlcustom.csv
200, save result to：gpt4mini/gpt-4.1-mini-salary_predictions_nohtmlcustom.csv
300, save result to：gpt4mini/gpt-4.1-mini-salary_predictions_nohtmlcustom.csv
400, save result to：gpt4mini/gpt-4.1-mini-salary_predictions_nohtmlcustom.csv
500, save result to：gpt4mini/gpt-4.1-mini-salary_predictions_nohtmlcustom.csv


In [8]:
correct

496

In [9]:
number

567

In [10]:
accuracy = correct / number
print(f"accuracy：{accuracy:.2%}")
print(f"save result to：{filename}")


accuracy：87.48%
save result to：gpt4mini/gpt-4.1-mini-salary_predictions_nohtmlcustom.csv
